In [2]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

2024-04-11 10:10:40.297973: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv('traindata2.csv')
df.head()

,redCorner,blueCorner,winner,referee,venue,title_fight,billing,redCorner_wins,blueCorner_wins,redCorner_losses,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,0,173,172,0,0,23,31,5,...,0,4.480000,4.960000,25.000000,4.960000,4.480000,0.485477,0.495495,1.000000,0.538462
1,0,2,2,26,172,0,1,22,14,6,...,3,2.133333,2.800000,15.000000,2.800000,2.133333,0.505882,0.632184,0.571429,1.000000
2,0,2,2,191,172,0,2,19,15,9,...,0,3.266667,4.733333,15.000000,4.733333,3.266667,0.481752,0.528846,0.875000,0.600000
3,0,2,0,88,172,0,2,14,20,5,...,0,11.214953,4.766355,3.566667,4.766355,11.214953,0.622222,0.411765,1.000000,1.000000
4,0,2,2,191,172,0,2,8,9,2,...,0,3.133333,3.600000,15.000000,3.600000,3.133333,0.460000,0.515464,0.600000,0.333333


In [4]:
dfInput = pd.read_csv('customInput.csv')
dfInput = dfInput.drop('Unnamed: 0', axis=1, errors='ignore')
dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,redCorner_fightTime,blueCorner_fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,Alex Pereira,Jamahal Hill,NaN,9,12,2,1,0,0,36,...,5.0,7.31,11.2,9.233333,3.65,3.35,0.5,0.47,0.7,0.73


In [5]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6268
6268


In [6]:
#dropping referee
df.drop('referee', axis=1, inplace=True)

#drop billings
df.drop('billing', axis=1, inplace=True)

#drop venue
df.drop('venue', axis=1, inplace=True)

#drop title_fight
df.drop('title_fight', axis=1, inplace=True)


df.head()


,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,0,23,31,5,10,0,0,28.0,...,0,4.480000,4.960000,25.000000,4.960000,4.480000,0.485477,0.495495,1.000000,0.538462
1,0,2,2,22,14,6,7,1,0,35.0,...,3,2.133333,2.800000,15.000000,2.800000,2.133333,0.505882,0.632184,0.571429,1.000000
2,0,2,2,19,15,9,6,1,0,28.0,...,0,3.266667,4.733333,15.000000,4.733333,3.266667,0.481752,0.528846,0.875000,0.600000
3,0,2,0,14,20,5,11,0,0,26.0,...,0,11.214953,4.766355,3.566667,4.766355,11.214953,0.622222,0.411765,1.000000,1.000000
4,0,2,2,8,9,2,1,0,0,28.0,...,0,3.133333,3.600000,15.000000,3.600000,3.133333,0.460000,0.515464,0.600000,0.333333


In [7]:
df.rename(columns={'fightTime': 'redCorner_fightTime'}, inplace=True)
df.insert(32, 'blueCorner_fightTime', df['redCorner_fightTime'])

dfColumns = df.columns
print(len(dfColumns))

39


In [7]:
dfInputColumns = dfInput.columns
print(len(dfInputColumns))

39


In [8]:
#one hot encode redCorner_nation
column = df[['redCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_nation
column = df[['blueCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
df = pd.concat([df, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_103,blueCorner_nation_104,blueCorner_nation_105,blueCorner_nation_106,blueCorner_nation_107,blueCorner_nation_108,blueCorner_nation_109,blueCorner_nation_110,blueCorner_nation_111,blueCorner_nation_112
0,0,2,0,23,31,5,10,0,0,28.0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,2,22,14,6,7,1,0,35.0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,2,19,15,9,6,1,0,28.0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,14,20,5,11,0,0,26.0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,2,8,9,2,1,0,0,28.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#one hot encode redCorner_stance
column = df[['redCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_stance'], prefix='redCorner_stance').astype(int)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_stance
column = df[['blueCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_stance'], prefix='blueCorner_stance').astype(int)
df = pd.concat([df, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,2,0,23,31,5,10,0,0,28.0,...,0,0,1,0,0,0,1,0,0,0
1,0,2,2,22,14,6,7,1,0,35.0,...,0,0,1,0,0,0,0,0,0,1
2,0,2,2,19,15,9,6,1,0,28.0,...,0,0,1,0,0,0,1,0,0,0
3,0,2,0,14,20,5,11,0,0,26.0,...,0,0,1,0,0,0,1,0,0,0
4,0,2,2,8,9,2,1,0,0,28.0,...,0,0,0,0,0,1,1,0,0,0


In [10]:
for index, row in df.iterrows():
    if row['winner'] == 1:
        df.drop(index)

df['blueCorner'] = 1
df['winner'] = df['winner'].replace(2, 1)


df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,1,0,23,31,5,10,0,0,28.0,...,0,0,1,0,0,0,1,0,0,0
1,0,1,1,22,14,6,7,1,0,35.0,...,0,0,1,0,0,0,0,0,0,1
2,0,1,1,19,15,9,6,1,0,28.0,...,0,0,1,0,0,0,1,0,0,0
3,0,1,0,14,20,5,11,0,0,26.0,...,0,0,1,0,0,0,1,0,0,0
4,0,1,1,8,9,2,1,0,0,28.0,...,0,0,0,0,0,1,1,0,0,0


In [11]:
#one hot encode redCorner_nation
column = df[['redCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
dfInput.drop('redCorner_nation', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

#one hot encode blueCorner_nation
column = df[['blueCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
dfInput.drop('blueCorner_nation', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

df.drop('redCorner_nation', axis=1, inplace=True)
df.drop('blueCorner_nation', axis=1, inplace=True)

dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_103,blueCorner_nation_104,blueCorner_nation_105,blueCorner_nation_106,blueCorner_nation_107,blueCorner_nation_108,blueCorner_nation_109,blueCorner_nation_110,blueCorner_nation_111,blueCorner_nation_112
0,Alex Pereira,Jamahal Hill,NaN,9.0,12.0,2.0,1.0,0.0,0.0,36.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#one hot encode redCorner_stance
column = df[['redCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_stance'], prefix='redCorner_stance').astype(int)
dfInput.drop('redCorner_stance', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

#one hot encode blueCorner_stance
column = df[['blueCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_stance'], prefix='blueCorner_stance').astype(int)
dfInput.drop('blueCorner_stance', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

df.drop('redCorner_stance', axis=1, inplace=True)
df.drop('blueCorner_stance', axis=1, inplace=True)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,1,0,23,31,5,10,0,0,28.0,...,0,0,1,0,0,0,1,0,0,0
1,0,1,1,22,14,6,7,1,0,35.0,...,0,0,1,0,0,0,0,0,0,1
2,0,1,1,19,15,9,6,1,0,28.0,...,0,0,1,0,0,0,1,0,0,0
3,0,1,0,14,20,5,11,0,0,26.0,...,0,0,1,0,0,0,1,0,0,0
4,0,1,1,8,9,2,1,0,0,28.0,...,0,0,0,0,0,1,1,0,0,0


In [15]:
#limit the data
dfInput = dfInput.head(1)
print(dfInput.iloc[0])

redCorner              Alex Pereira
blueCorner             Jamahal Hill
winner                          NaN
redCorner_wins                  9.0
blueCorner_wins                12.0
                           ...     
redCorner_stance_3                0
blueCorner_stance_0               1
blueCorner_stance_1               0
blueCorner_stance_2               0
blueCorner_stance_3               0
Name: 0, Length: 250, dtype: object


In [16]:
dfColumns = df.columns
for column in dfColumns:
    print(column)

redCorner
blueCorner
winner
redCorner_wins
blueCorner_wins
redCorner_losses
blueCorner_losses
redCorner_draws
blueCorner_draws
redCorner_age
blueCorner_age
redCorner_knockdowns
blueCorner_knockdowns
redCorner_sig_str_percentage
blueCorner_sig_str_percentage
redCorner_takedowns
blueCorner_takedowns
redCorner_takedown_percentage
blueCorner_takedown_percentage
redCorner_subs_attempted
blueCorner_subs_attempted
redCorner_height
blueCorner_height
redCorner_reach
blueCorner_reach
redCorner_sig_str_landed_per_minute
blueCorner_sig_str_landed_per_minute
fightTime
redCorner_sig_str_absorbed_per_minute
blueCorner_sig_str_absorbed_per_minute
redCorner_sig_str_defense
blueCorner_sig_str_defense
redCorner_takedown_defense
blueCorner_takedown_defense
redCorner_nation_0
redCorner_nation_1
redCorner_nation_2
redCorner_nation_3
redCorner_nation_4
redCorner_nation_5
redCorner_nation_6
redCorner_nation_7
redCorner_nation_8
redCorner_nation_9
redCorner_nation_10
redCorner_nation_11
redCorner_nation_12
red

In [17]:
dfInputColumns = dfInput.columns
for column in dfInputColumns:
    print(column)

redCorner
blueCorner
winner
redCorner_wins
blueCorner_wins
redCorner_losses
blueCorner_losses
redCorner_draws
blueCorner_draws
redCorner_age
blueCorner_age
redCorner_knockdowns
blueCorner_knockdowns
redCorner_sig_str_percentage
blueCorner_sig_str_percentage
redCorner_takedowns
blueCorner_takedowns
redCorner_takedown_percentage
blueCorner_takedown_percentage
redCorner_subs_attempted
blueCorner_subs_attempted
redCorner_height
blueCorner_height
redCorner_reach
blueCorner_reach
redCorner_sig_str_landed_per_minute
blueCorner_sig_str_landed_per_minute
redCorner_fightTime
blueCorner_fightTime
redCorner_sig_str_absorbed_per_minute
blueCorner_sig_str_absorbed_per_minute
redCorner_sig_str_defense
blueCorner_sig_str_defense
redCorner_takedown_defense
blueCorner_takedown_defense
redCorner_nation_0
redCorner_nation_1
redCorner_nation_2
redCorner_nation_3
redCorner_nation_4
redCorner_nation_5
redCorner_nation_6
redCorner_nation_7
redCorner_nation_8
redCorner_nation_9
redCorner_nation_10
redCorner_na

In [18]:
#when one hot encoding, nations that have no representation in the train set have Nan values. This sets them to zero as they would have been zero if they were represented
df.fillna(0, inplace=True)

In [19]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6268
6268


In [20]:
print(len(dfInput))
dfInput.dropna(subset=dfInput.columns.difference(['winner']), inplace=True)
print(len(dfInput))

1
1


In [21]:
dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,Alex Pereira,Jamahal Hill,NaN,9.0,12.0,2.0,1.0,0.0,0.0,36.0,...,0,0,1,0,0,0,1,0,0,0


In [22]:
#x,y split
target_column = 'winner'
y = df[target_column]
X = df.drop(target_column, axis=1)

#create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





In [25]:
#Define the deep learning model
model = Sequential()
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

#earlyStopping
earlystopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath='best_model_weights2.weights.h5', save_weights_only=True, monitor='val_accuracy', save_best_only=True, verbose=1)

#compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train
model.fit(X_train, y_train, epochs=1000, batch_size=124, validation_data=(X_test, y_test), callbacks=[checkpoint, earlystopping])

Epoch 1/1000
19/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5674 - loss: 0.7607 
Epoch 1: val_accuracy improved from -inf to 0.77512, saving model to best_model_weights2.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6126 - loss: 0.6939 - val_accuracy: 0.7751 - val_loss: 0.4743
Epoch 2/1000
37/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7944 - loss: 0.4529
Epoch 2: val_accuracy improved from 0.77512 to 0.82935, saving model to best_model_weights2.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7960 - loss: 0.4506 - val_accuracy: 0.8293 - val_loss: 0.3986
Epoch 3/1000
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8346 - loss: 0.3915
Epoch 3: val_accuracy improved from 0.82935 to 0.84211, saving model to best_model_weights2.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8345 - loss: 0.3914 - val_accuracy: 0.8421 - val_loss: 0.3674
Epoch 4/1000
24/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8453 - loss: 0.3659
Epo

In [26]:
for index, row in dfInput.iterrows():
    dfInput.at[index, 'redCorner'] = 0.0
    dfInput.at[index, 'blueCorner'] = 1.0

dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_111,blueCorner_nation_112,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0.0,1.0,NaN,9.0,12.0,2.0,1.0,0.0,0.0,36.0,...,0,0,1,0,0,0,1,0,0,0


In [27]:
dfInput = dfInput.astype(float)

In [28]:
#accuracy
y_hat = model.predict(X_test)
y_hat = [0 if val<0.5 else 1 for val in y_hat]
accuracy_score(y_test,y_hat)

#0.8545897644191714

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


0.8644338118022329

In [30]:
#testing the model using career statistics

# x, y split for dfMock
target_column = 'winner'
y_mock = dfInput[target_column]
X_mock = dfInput.drop(target_column, axis=1)

#accuracy
y_hat = model.predict(X_mock)
print(y_hat)




2024-04-11 10:16:00.778300: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Matrix size-incompatible: In[0]: [1,249], In[1]: [248,64]
	 [[{{node sequential_2_1/dense_8_1/MatMul}}]]


InvalidArgumentError: Graph execution error:

Detected at node sequential_2_1/dense_8_1/MatMul defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/maxbraithwaite/Desktop/UFCModel/UFCModel/UFCModelGit/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/maxbraithwaite/Desktop/UFCModel/UFCModel/UFCModelGit/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/maxbraithwaite/Desktop/UFCModel/UFCModel/UFCModelGit/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/maxbraithwaite/Desktop/UFCModel/UFCModel/UFCModelGit/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/Library/Fr